In [1]:
# importing all required packages/libraries
import requests
import pandas as pd
import numpy as np
import re
import string
import json

In [2]:
# file paths
QUERIES_FP = 'query.ohsu.1-63'
JUDGEMENTS_FP = 'qrels.ohsu.87-91.merged.txt'
DOCUMENTS1_FP = 'ohsumed-87.json'
DOCUMENTS2_FP = 'ohsumed-88-91.json'

In [3]:
# to fetch queries from QUERIES_FP into a dictionary
# code reference : https://github.com/soldni/elasticsearch-demo/blob/274133afd000406d3e09111823413fe0f862acc3/search.py
def parse_raw_queries(raw_data):
    
    queries = []
    
    raw_queries = re.split(r'</top>\s*<top>', raw_data)
    
    # get rid of start and end <top> tags
    raw_queries[0] = raw_queries[0].replace(r'<top>','')
    raw_queries[len(raw_queries)-1] = raw_queries[len(raw_queries)-1].replace(r'</top>','')
    #print(raw_queries)
    
    for raw_query in raw_queries:
        # find the query identifier
        q_id = re.search(r'<num> Number: (OHSU\d+)', raw_query).group(1)
        #print(q_id)
        
        # find the title of the query
        q_title = re.search(r'<title> (.+)', raw_query).group(1)
        #print(q_title)
        
        # fetch the query
        q_desc = re.search(r'<desc> Description:\W*\n([\w\W]+)', raw_query)\
                .group(1).replace('\n', ' ').strip()
        #print(q_desc)
        
        queries.append({'_id': q_id, 'title': q_title, 'desc': q_desc})
    
    #print(queries)
    return queries

In [4]:
# invoke the query parser
with open(QUERIES_FP) as f:
    topics = parse_raw_queries(f.read())

In [5]:
# view parsed data
topics[:5]

[{'_id': 'OHSU1',
  'title': '60 year old menopausal woman without hormone replacement therapy',
  'desc': 'Are there adverse effects on lipids when progesterone is given with estrogen replacement therapy'},
 {'_id': 'OHSU2',
  'title': '60 yo male with disseminated intravascular coagulation',
  'desc': 'pathophysiology and treatment of disseminated intravascular coagulation'},
 {'_id': 'OHSU3',
  'title': 'prolonged prothrombin time',
  'desc': 'anticardiolipin and lupus anticoagulants, pathophysiology, epidemiology, complications'},
 {'_id': 'OHSU4',
  'title': '58 yo with cancer and hypercalcemia',
  'desc': 'effectiveness of etidronate in treating hypercalcemia of malignancy'},
 {'_id': 'OHSU5',
  'title': '55 yo female,postmenopausal',
  'desc': 'does estrogen replacement therapy cause breast cancer'}]

In [6]:
# how to fetch queries from list of dict
only_queries = [t['desc'] for t in topics]
only_queries[:5]

['Are there adverse effects on lipids when progesterone is given with estrogen replacement therapy',
 'pathophysiology and treatment of disseminated intravascular coagulation',
 'anticardiolipin and lupus anticoagulants, pathophysiology, epidemiology, complications',
 'effectiveness of etidronate in treating hypercalcemia of malignancy',
 'does estrogen replacement therapy cause breast cancer']

In [8]:
%%time
# send each query to SOLR and fetch top k results for each query in a dict
# note we use ohsumed_test2
base_url = 'http://localhost:8983/solr/ohsumed_test2/select?'
#CHANGED
k = 10
topk_results = {}
for t in topics:
    query = t['desc']
    params = {
    'indent': 'true',
    'fl': 'medline_ui',
    'q.op': 'OR',
    'rows': k,
    'q': query
    }

    r = requests.get(base_url, params=params)
    l = [r.json()['response']['docs'][i]['medline_ui'][0] for i in range(k)]
    #print(l)
    
    topk_results[t['_id']] = l

Wall time: 2min 13s


In [9]:
#view result
topk_results['OHSU1']

[90350732,
 89244872,
 90377812,
 90010830,
 90191478,
 91310766,
 91226903,
 91057187,
 91261347,
 87316316]

In [10]:
#fetching only 'definitely relevant' documents for each query
q_ids =  [t['_id'] for t in topics]
#print(q_ids)
judgements_list = []
d_relevant = {}
with open(JUDGEMENTS_FP) as f:
    for l in f.readlines():
        judgements_list.append(l.split())
    

In [11]:
# filtering for 2 (definitely relevant) documents for each query
for q_id in q_ids:
    d_rel = [int(j[1]) for j in judgements_list if j[0] == q_id and j[2] == '2']
    d_relevant[q_id] = d_rel

In [13]:
#view data
d_relevant['OHSU1'] 

[87157536,
 87157537,
 87097544,
 91336317,
 91185608,
 89093443,
 89270656,
 89244872,
 88095668,
 91110205,
 89182887,
 90135739,
 90191478,
 89225293,
 88238705,
 88238706,
 88196514,
 89201699,
 90278894,
 89228983,
 89201700,
 91120636,
 91120637,
 91229114]

In [14]:
# getting the count of d_rel docs for each query
d_rel_count = {}
for key,val in d_relevant.items():
    d_rel_count[key] = len(val)
    

In [15]:
#IMP data
d_rel_count

{'OHSU1': 24,
 'OHSU2': 11,
 'OHSU3': 118,
 'OHSU4': 18,
 'OHSU5': 30,
 'OHSU6': 5,
 'OHSU7': 12,
 'OHSU8': 10,
 'OHSU9': 23,
 'OHSU10': 15,
 'OHSU11': 76,
 'OHSU12': 5,
 'OHSU13': 46,
 'OHSU14': 28,
 'OHSU15': 13,
 'OHSU16': 26,
 'OHSU17': 14,
 'OHSU18': 63,
 'OHSU19': 36,
 'OHSU20': 7,
 'OHSU21': 49,
 'OHSU22': 7,
 'OHSU23': 35,
 'OHSU24': 29,
 'OHSU25': 9,
 'OHSU26': 31,
 'OHSU27': 4,
 'OHSU28': 12,
 'OHSU29': 63,
 'OHSU30': 94,
 'OHSU31': 24,
 'OHSU32': 29,
 'OHSU33': 83,
 'OHSU34': 11,
 'OHSU35': 79,
 'OHSU36': 47,
 'OHSU37': 52,
 'OHSU38': 32,
 'OHSU39': 96,
 'OHSU40': 39,
 'OHSU41': 6,
 'OHSU42': 27,
 'OHSU43': 21,
 'OHSU44': 30,
 'OHSU45': 7,
 'OHSU46': 14,
 'OHSU47': 37,
 'OHSU48': 45,
 'OHSU49': 49,
 'OHSU50': 26,
 'OHSU51': 50,
 'OHSU52': 8,
 'OHSU53': 6,
 'OHSU54': 10,
 'OHSU55': 26,
 'OHSU56': 13,
 'OHSU57': 15,
 'OHSU58': 6,
 'OHSU59': 34,
 'OHSU60': 14,
 'OHSU61': 15,
 'OHSU62': 12,
 'OHSU63': 49}

In [16]:
# marking definitely relevent docs as 1 and rest as 0 for every query's top_k

relevant_results = {}

for q_id in q_ids:
    topk = topk_results[q_id]
    def_rel = d_relevant[q_id]
    x = [1 if k in def_rel else 0 for k in topk ]
    
    
    relevant_results[q_id] = x

#relevant_results


In [18]:
#Calculating avg_precision@k for every query
avg_precision = []
rel_list_to_file = []
#for every query
for key,val in relevant_results.items():
    #print("rel list : " , val)
    rel_list_to_file.append(val)
    #print("Num. of rel docs returned: ",sum(val))
    #print("Actual no of rel. docs: ",d_rel_count[key])
    total = 0;
    # for every doc(k)
    for i,v in enumerate(val):
        
        if v == 1:
            #print("index: ",i+1," value :",v)
            #print("Precision at index",i+1," : ",sum(val[:i+1])/(i+1))
            total += sum(val[:i+1])/(i+1)   
    if sum(val)!=0:    #handling div by 0 error
        #print("Avg precision : ",total/sum(val),"\n")
        avg_precision.append(total/sum(val))
    else:
        avg_precision.append(0)
  
    

In [19]:
#MAP@k
map = sum(avg_precision)/len(avg_precision)
map

0.48580147182329725

In [27]:
#viewing all the avg_p

#for i,p in enumerate(avg_precision):
    #print(i+1, " ",p)

In [120]:
'''RUN ONLY FOR WHEN YOU HAVE RESULTS FOR TOP_K FOR K = 50 '''
#NEW
#write the relevant docs in top k to file for R script to read
#rel_list_to_file
df_rel_list_to_file = pd.DataFrame(np.array(rel_list_to_file))
pd.set_option('display.max_columns', None)
#df_rel_list_to_file
df_rel_list_to_file.to_csv('relevant_doc_list_top_50_for_63q.csv', index=False,header=False)

In [32]:
#NEW
# reading new_rel docs and calculating avg. p@10
new_rel_df = pd.read_csv('new_relevant_docs_top_10.csv',header = None)
#new_rel_df

#make a list of lists
new_rel_list = list()
for row in new_rel_df.index:
    #print(list(new_rel_df.loc[row]))
    new_rel_list.append(list(new_rel_df.loc[row]))
    
new_rel_list


[[0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0, 1, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 0, 1, 0, 1, 0, 0, 1, 1],
 [0, 0, 0,

In [29]:
#NEW
#Calculating avg_precision for new result set and MAP

avg_precision_new = []

#for every query
for val in new_rel_list:
    #print("rel list : " , val)
    #print("Num. of rel docs returned: ",sum(val))
    #print("Actual no of rel. docs: ",d_rel_count[key])
    total = 0;
    # for every doc(k)
    for i,v in enumerate(val):
        
        if v == 1:
            #print("index: ",i+1," value :",v)
            #print("Precision at index",i+1," : ",sum(val[:i+1])/(i+1))
            total += sum(val[:i+1])/(i+1)   
    if sum(val)!=0:    #handling div by 0 error
        #print("Avg precision : ",total/sum(val),"\n")
        avg_precision_new.append(total/sum(val))
    else:
        avg_precision_new.append(0)
    

 

In [30]:
# new MAP@k
map_new = sum(avg_precision_new)/len(avg_precision_new)
map_new

0.5087650721664329

In [31]:
#viewing all the avg_p
#for i,p in enumerate(avg_precision_new):
    #print(p)

### Finding similarity between top k results

In [22]:
# fetching the documents to be used as queries 
f = open(DOCUMENTS1_FP, 'r')
data1 = json.loads(f.read())

In [23]:
f = open(DOCUMENTS2_FP, 'r')
data2 = json.loads(f.read())

In [24]:
data = data1 + data2

In [25]:
#IMP data
index_data = {}
#chars = re.escape('"#$%&\'()*+/:;<=>@[\\]^_`{|}~')
chars = re.escape('"#$%&\'()*+-/:;<=>@[\\]^_`{|}~')
for d in data:
    temp_str = '; '.join(d['mesh_terms']) + " " + d['title'] + " " + d['abstract'] + " " +'; '.join(d['publication_type']) + " " +'; '.join(d['author']) + " " + d['source']
    #temp_str = temp_str.replace(':','')
    temp_str = re.sub(r'['+chars+']', '', temp_str)
    #print(temp_str)
    index_data[d['medline_ui']] = temp_str

In [26]:
# dict with key:medline_ui and value:6 concatenated index fields
len(index_data)   #348566

348566

In [136]:
result_q1 = topk_results['OHSU6']
result_q1

[90096304,
 89279941,
 88304562,
 90332112,
 91378768,
 87155655,
 91082911,
 87242941,
 90063654,
 90010623,
 89038754,
 87301271,
 89365600,
 90040519,
 91149035,
 91290853,
 87287799,
 90124165,
 88109230,
 89274990,
 87153457,
 89002611,
 91129334,
 91059321,
 90317642,
 91055124,
 90274051,
 90358174,
 91373619,
 90266802,
 90268107,
 88253106,
 89299651,
 89010876,
 90203331,
 89054525,
 88228706,
 88279675,
 91241805,
 90002756,
 91058143,
 91051601,
 88222416,
 91064692,
 89089494,
 87158084,
 91365916,
 91003852,
 88023283,
 91309029]

In [123]:
%%time   

# send each result in top_k as query to SOLR and fetch top m results in a dict
# note we use ohsumed_test2

m = 1000
similarity_lists = []

#for med_ui in result_q1 :
for med_ui in result_q1:
    temp = {}
    temp_l = []
    query = index_data[med_ui]
    params = {
    'indent': 'true',
    'fl': ['medline_ui', 'score'],
    'q.op': 'OR',
    'rows': m,
    'q': query
    }

    response = requests.get(base_url, params=params)
    #print(r.json())
    
    # storing the score by looping over m results
    if response.json()['response']['numFound'] < m:
        for i in range(response.json()['response']['numFound']):
            temp[response.json()['response']['docs'][i]['medline_ui'][0]] = response.json()['response']['docs'][i]['score']
    else:    
        for i in range(m):
            temp[response.json()['response']['docs'][i]['medline_ui'][0]] = response.json()['response']['docs'][i]['score']
    
    print(temp)
    
    # fetching the scores of top_k results of the query
    for r in result_q1:
        if r in temp.keys():
            print(r," ",temp[r])
            temp_l.append(temp[r])
        else:
            print(r," ",0)
            temp_l.append(0)
            
    # appending into a list
    similarity_lists.append(temp_l)
    
similarity_lists
    
    

{91367211: 524.2435, 91147584: 205.97134, 91148654: 193.04198, 89297287: 191.54103, 89382494: 188.28969, 90240572: 185.19998, 91204017: 181.39001, 91260793: 178.51363, 89070633: 175.5589, 91232771: 174.39406, 89349262: 171.11464, 89017746: 170.7862, 87116813: 168.62688, 89005792: 165.27237, 91015202: 164.85896, 88091264: 164.4658, 89047485: 163.64124, 91128660: 160.90437, 89002013: 159.7244, 88271289: 158.28123, 90226423: 158.19522, 87270898: 156.79782, 90191446: 156.65364, 91223700: 156.47276, 90137673: 156.35771, 91339116: 155.99469, 88172344: 155.62558, 90144789: 155.2749, 89143658: 153.36278, 89071111: 153.29285, 91367230: 152.73637, 87283613: 151.7116, 90350721: 150.90047, 89365076: 149.57599, 90268153: 149.43596, 88276042: 149.33849, 89344137: 149.15295, 90328184: 149.14432, 89011727: 147.79306, 89320522: 147.71207, 90196081: 147.58391, 91166018: 146.43837, 88105911: 145.53325, 90309930: 145.35124, 91278237: 144.83635, 90328200: 144.68524, 88192725: 144.3838, 91367229: 144.22092,

{87325763: 208.02394, 88071561: 91.25417, 90340483: 87.98821, 90024936: 87.04021, 90240572: 82.1778, 87212721: 77.950935, 89190711: 77.10289, 88202437: 77.07369, 90340484: 75.49189, 90372674: 74.871544, 88091264: 74.309845, 87116813: 73.816414, 87311483: 69.905594, 91367211: 69.86805, 89349262: 69.62979, 89190713: 69.48891, 91128058: 69.18573, 87121335: 68.34304, 88035896: 68.0625, 89193854: 68.037094, 90141729: 66.96716, 90043189: 66.871284, 88172345: 66.60739, 91280626: 66.457695, 91295108: 66.21381, 90350721: 65.710396, 89271668: 65.70344, 90328190: 64.85276, 88071562: 64.84412, 89278386: 64.79422, 90049456: 64.69599, 87213431: 64.61971, 87270898: 64.082886, 91189191: 63.962166, 87283613: 62.815403, 89068606: 62.788563, 91096830: 62.30311, 91232771: 62.228954, 91147584: 62.19685, 89296241: 62.1965, 88172346: 62.154015, 89206454: 61.79317, 91118095: 61.258305, 89146778: 61.212082, 88286584: 61.1906, 90162773: 61.161716, 90371555: 60.841644, 89292205: 60.75163, 89297287: 60.41716, 872

{88230682: 385.15573, 89157633: 151.44528, 90030861: 147.64148, 89259188: 129.6263, 91273412: 124.152084, 89190711: 118.3303, 90047453: 117.01195, 91306954: 116.466606, 90144789: 115.5027, 88173617: 114.85974, 90372674: 113.79935, 88091264: 113.414665, 89148575: 113.147255, 87154817: 113.032074, 88321989: 112.180725, 87124934: 112.154854, 88236200: 111.296646, 90294406: 110.55934, 89141973: 110.252464, 90262944: 109.60863, 88230693: 109.54193, 88215074: 109.3888, 90340483: 109.364555, 91226039: 108.625755, 90054678: 107.4757, 90376563: 106.92385, 89349262: 106.498245, 90112688: 106.46828, 90112694: 106.06469, 89093202: 106.05552, 89133733: 105.844406, 88009133: 104.92811, 91052125: 104.81396, 89146778: 104.57452, 87261912: 104.43669, 88233213: 104.34613, 90053291: 104.22582, 89141972: 104.222824, 88277984: 104.04968, 87110264: 103.90647, 90296248: 103.374245, 88333247: 103.21321, 90365143: 102.933235, 90012440: 102.804016, 87250730: 102.733604, 87120742: 102.5943, 88337035: 102.49902, 

{90191446: 349.02902, 89017746: 119.93325, 88103813: 117.20413, 89349262: 114.31882, 91306954: 113.68402, 91074460: 113.31606, 91128040: 110.80622, 90350695: 109.47994, 90221364: 106.64007, 89103399: 106.207146, 90114837: 105.71818, 90386269: 104.322586, 88276414: 103.03989, 90297108: 102.59562, 89270245: 102.36464, 89297287: 101.75783, 91296298: 100.61354, 90099750: 100.58229, 88024864: 100.52637, 89263470: 99.25772, 91255762: 99.24482, 88279614: 98.97118, 88001814: 98.42, 90318677: 98.20339, 90225070: 97.63947, 89142035: 97.36365, 90045256: 97.310425, 91147584: 97.28799, 88074540: 97.147095, 90072304: 96.98849, 87202729: 96.68817, 91232766: 96.500656, 88251222: 95.982155, 90158706: 95.971016, 88192725: 95.78318, 91333899: 95.7081, 90221358: 95.674286, 91241190: 94.95171, 87116813: 94.70875, 91165901: 94.62045, 90283598: 94.334595, 89390466: 93.93778, 91044014: 93.923386, 91232767: 93.77373, 90386227: 93.72075, 88250307: 93.57816, 91039081: 93.44674, 91118095: 93.44461, 91194894: 93.4

{87242958: 425.49622, 87283601: 157.41289, 88294302: 153.77696, 88091264: 145.91646, 90144789: 141.73782, 90350721: 141.08403, 91374975: 140.56822, 88089013: 129.37718, 91128098: 128.35777, 89208066: 127.79151, 88009133: 127.595146, 89349262: 127.153305, 90266791: 127.0475, 87259250: 124.48611, 91147584: 123.27727, 90028785: 121.941635, 91096830: 121.609924, 91367211: 120.613045, 87264624: 119.94455, 90286762: 118.28177, 90030861: 118.17656, 87254081: 118.09012, 87325485: 117.38687, 90355087: 116.50196, 90073179: 116.47702, 90272664: 116.16549, 89292205: 115.41199, 89133733: 114.32799, 91143675: 113.3169, 88277984: 112.1971, 88286584: 111.84684, 87154817: 111.09311, 89190711: 110.41457, 90226423: 110.21274, 91195293: 110.14579, 90159410: 109.360115, 91295108: 108.70219, 90139446: 108.55585, 90187150: 107.42094, 87311483: 107.37858, 89015716: 107.33188, 89146778: 106.48972, 90240572: 105.83062, 88331389: 104.56841, 88230682: 104.482735, 89206454: 104.14687, 89350344: 104.07042, 91322623

...

{90088631: 64.92994, 90355689: 34.679092, 89011763: 34.46926, 91273655: 34.26205, 88149707: 34.137894, 89061796: 33.949024, 91025167: 33.669575, 88192761: 33.475834, 91083721: 33.381313, 90355086: 33.118435, 89236256: 32.607735, 88108432: 32.580086, 87298647: 32.5283, 91191298: 32.123226, 90384270: 31.830309, 88340096: 31.740347, 87184742: 31.68822, 90226441: 31.664555, 91155636: 31.476604, 88024294: 31.271805, 87100320: 31.255615, 90166063: 31.209974, 87213431: 31.10596, 88162961: 30.895624, 87270899: 30.852226, 88340093: 30.79638, 90266860: 30.7212, 89351106: 30.62232, 87184740: 30.525518, 91083703: 30.062931, 91083708: 30.05467, 88309191: 29.995152, 89050370: 29.955355, 88134441: 29.867128, 90062670: 29.814655, 90088626: 29.813095, 88240609: 29.77733, 87212094: 29.576143, 88340095: 29.479095, 88134438: 29.476498, 90139434: 29.461876, 89374382: 29.436476, 91166604: 29.427483, 88049849: 29.412052, 88077224: 29.217634, 88340098: 29.211864, 87156875: 29.081417, 88240621: 29.081417, 8937

{87283604: 369.47626, 87097495: 177.15247, 88286584: 131.59258, 90350721: 128.95374, 90262944: 121.42034, 89349262: 120.41628, 87270899: 119.06315, 89068606: 118.18006, 87213431: 116.66418, 89193854: 116.55913, 89297287: 116.21067, 90056636: 111.97556, 90240572: 109.67968, 88071562: 109.634514, 90204393: 109.372406, 89382492: 108.7093, 89157470: 105.35919, 91006596: 105.325005, 89380927: 105.222595, 89011727: 105.18284, 90026453: 105.08358, 87298647: 104.99579, 87270898: 104.98474, 88192725: 104.53054, 90144789: 103.9889, 88172346: 102.79821, 90002058: 102.643074, 89328906: 102.10389, 87242958: 102.06336, 88161239: 102.031525, 89107592: 101.20945, 91147584: 100.94887, 90267539: 100.55849, 87311483: 100.29893, 88245165: 99.7083, 88240610: 99.1492, 87198514: 98.20918, 89047485: 97.47086, 89382494: 97.36437, 90322559: 97.352875, 90023527: 97.21799, 88192743: 96.9509, 89157486: 96.71988, 89208066: 96.373566, 88035873: 95.96183, 89258659: 95.910515, 90379201: 95.797226, 90266791: 95.68532, 

{87155853: 171.84706, 89236234: 65.06337, 87128270: 62.929787, 89017746: 61.851086, 89380927: 59.950695, 89297287: 57.70661, 91118095: 56.67857, 88105911: 51.20406, 88302917: 50.999813, 91147584: 50.94982, 89317660: 49.86327, 89382896: 48.93495, 89344769: 48.676006, 91367211: 48.186024, 91260793: 48.138893, 87212094: 47.94134, 89011727: 46.978794, 91291007: 46.45846, 87116813: 46.24469, 90191446: 46.136414, 91016198: 46.0666, 91333900: 45.757946, 88103813: 45.619823, 89349262: 44.79065, 89163844: 44.6858, 89352378: 44.3953, 91086899: 44.21622, 88169955: 44.10762, 88192725: 43.91754, 87198490: 43.90317, 91127488: 43.750027, 87283613: 43.713795, 88250666: 43.648502, 88214988: 43.31637, 87325051: 43.24095, 89236256: 43.12487, 91196776: 43.029236, 87198514: 43.02308, 89270659: 42.993797, 90144575: 42.852337, 87201789: 42.53511, 91022863: 42.434086, 90043189: 42.3662, 87270899: 42.30381, 91368849: 42.278744, 91320857: 42.260944, 91178477: 42.150707, 90321893: 42.13741, 88161230: 42.003384, 

{88240609: 396.0752, 91190163: 163.90717, 90137283: 120.78851, 91096831: 117.523384, 88162938: 116.39677, 90303354: 116.02535, 88035877: 115.440315, 89017739: 111.70574, 90240572: 108.05101, 90322559: 104.702965, 90189079: 104.49786, 88286582: 103.367355, 90204393: 103.052704, 87213431: 102.87839, 90023527: 101.25146, 88035873: 101.059326, 90226423: 100.91357, 89297287: 100.30505, 87311483: 100.0918, 87311484: 97.23685, 88192754: 96.0843, 87283604: 95.317276, 89193854: 94.98381, 89380927: 94.45651, 87298647: 93.61302, 90197679: 93.04607, 87167553: 92.95262, 89068606: 92.5167, 89374372: 92.505226, 87270898: 92.32986, 90204395: 91.84635, 90002058: 91.38444, 89311273: 91.20514, 91374406: 91.12217, 91218111: 90.76502, 91218110: 90.6696, 88286584: 90.635796, 87145052: 90.466545, 90055536: 90.458176, 89206833: 90.357506, 90088619: 90.18368, 89149836: 90.165276, 91311597: 89.91436, 87258100: 89.79672, 88192725: 89.7196, 87154817: 89.63854, 89303436: 89.50374, 89382492: 89.47808, 89225915: 89.

[[524.2435,
  119.128685,
  117.799095,
  156.65364,
  135.67642,
  124.972145,
  129.55562,
  0,
  171.11464,
  191.54103,
  105.29632,
  205.97134,
  117.72742,
  164.4658,
  170.7862,
  0,
  147.79306,
  127.51883,
  0,
  0,
  132.86726,
  95.80108,
  130.37733,
  107.61035,
  97.04229,
  100.70107,
  0,
  185.19998,
  99.15861,
  0,
  112.295006,
  137.98181,
  105.082634,
  156.79782,
  115.87168,
  151.7116,
  96.18755,
  98.08112,
  0,
  110.225555,
  0,
  150.90047,
  99.69609,
  126.07368,
  133.40532,
  118.34165,
  0,
  95.84704,
  109.378654,
  0],
 [69.86805,
  208.02394,
  56.35578,
  48.974556,
  56.299946,
  48.970676,
  61.212082,
  42.180847,
  69.62979,
  60.41716,
  49.234444,
  62.19685,
  54.99666,
  74.309845,
  45.601486,
  38.80512,
  59.611156,
  66.21381,
  43.656536,
  58.94668,
  62.30311,
  91.25417,
  42.39637,
  49.400467,
  45.990746,
  64.84412,
  45.49195,
  82.1778,
  60.01361,
  0,
  64.61971,
  57.28807,
  87.04021,
  64.082886,
  60.75163,
  62.81

In [124]:
# storing in a df

#df = pd.DataFrame(np.array(similarity_lists),columns = result_q1, index = result_q1)
df = pd.DataFrame(np.array(similarity_lists))
pd.set_option('display.max_columns', None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,524.243500,119.128685,117.799095,156.653640,135.676420,124.972145,129.555620,0.000000,171.114640,191.541030,105.296320,205.971340,117.727420,164.465800,170.786200,0.000000,147.793060,127.518830,0.000000,0.000000,132.867260,95.801080,130.377330,107.610350,97.042290,100.701070,0.000000,185.199980,99.158610,0.000000,112.295006,137.981810,105.082634,156.797820,115.871680,151.711600,96.187550,98.081120,0.000000,110.225555,0.000000,150.900470,99.696090,126.073680,133.405320,118.341650,0.000000,95.847040,109.378654,0.000000
1,69.868050,208.023940,56.355780,48.974556,56.299946,48.970676,61.212082,42.180847,69.629790,60.417160,49.234444,62.196850,54.996660,74.309845,45.601486,38.805120,59.611156,66.213810,43.656536,58.946680,62.303110,91.254170,42.396370,49.400467,45.990746,64.844120,45.491950,82.177800,60.013610,0.000000,64.619710,57.288070,87.040210,64.082886,60.751630,62.815403,37.810480,44.287132,61.161716,40.624977,41.229550,65.710396,45.807713,45.811325,36.169575,45.689705,39.466034,45.173897,41.731460,39.867870
2,82.809470,0.000000,385.155730,99.878890,90.916016,147.641480,104.574520,0.000000,106.498245,81.289280,0.000000,99.005180,0.000000,113.414665,98.139030,0.000000,0.000000,92.006090,0.000000,93.383260,92.506390,101.012215,116.466606,0.000000,0.000000,0.000000,0.000000,96.360790,91.999176,0.000000,79.631430,113.032074,0.000000,93.040060,95.293260,91.831390,0.000000,0.000000,0.000000,0.000000,0.000000,87.961580,81.762110,109.608630,85.709350,0.000000,0.000000,0.000000,0.000000,0.000000
3,87.526760,0.000000,78.008820,349.029020,0.000000,0.000000,69.694695,0.000000,114.318820,101.757830,0.000000,97.287990,0.000000,0.000000,119.933250,0.000000,73.996086,0.000000,82.489640,0.000000,0.000000,0.000000,113.684020,77.205530,0.000000,0.000000,0.000000,72.830580,0.000000,0.000000,68.255730,73.340576,0.000000,90.589070,71.385445,70.109830,0.000000,82.639660,0.000000,76.822815,0.000000,72.903656,0.000000,70.270294,0.000000,0.000000,0.000000,68.487040,67.978640,0.000000
4,120.613045,101.646660,104.482735,78.210106,425.496220,118.176560,106.489720,0.000000,127.153305,79.890620,117.386870,123.277270,71.939570,145.916460,73.192350,68.923256,80.309810,108.702190,91.339134,0.000000,121.609924,99.796170,92.600136,0.000000,91.526910,95.165690,0.000000,105.830620,119.944550,70.920840,80.335860,111.093110,75.086560,102.638940,115.411990,89.455340,116.477020,71.077730,0.000000,72.126770,76.835724,141.084030,66.290300,102.393950,85.416320,128.357770,0.000000,100.563550,0.000000,0.000000
5,85.049515,0.000000,121.433945,69.451065,99.044100,355.827240,102.167760,0.000000,91.999830,69.675000,0.000000,84.890350,75.654240,95.172150,84.991470,0.000000,68.110900,95.464714,0.000000,82.943214,104.576710,80.539240,75.147030,0.000000,0.000000,72.356660,0.000000,85.252100,81.806990,0.000000,81.291250,99.844930,0.000000,81.216110,0.000000,0.000000,0.000000,0.000000,69.183190,80.007690,0.000000,82.370125,84.806170,100.171020,0.000000,0.000000,0.000000,72.388710,0.000000,71.267380
6,86.737190,76.009010,86.629870,66.198654,87.950700,107.495040,279.598270,0.000000,103.637190,68.986930,83.358870,81.101360,88.946610,88.537620,74.675156,0.000000,80.694890,59.194160,62.757270,80.822075,90.135140,95.488450,83.068320,66.174430,65.254220,90.174370,0.000000,82.263230,82.730570,0.000000,86.256165,91.333230,75.845750,86.797020,78.524640,0.000000,0.000000,0.000000,94.282490,61.315132,0.000000,94.741104,73.287000,100.006874,95.523350,75.158440,0.000000,72.238030,0.000000,66.663980
7,51.067360,36.180565,36.235220,47.679480,0.000000,32.014540,31.860826,157.638500,58.677216,70.587210,0.000000,54.557300,42.893890,33.374302,56.018147,40.120350,61.190910,34.940952,47.271145,51.679030,35.503044,47.972454,48.056950,59.400333,43.145737,46.563816,46.082146,50.031097,28.421734,32.974358,44.630090,52.959743,0.000000,59.604256,35.069510,55.038822,0.0

In [125]:
#num of zeros in above df
num = 0
for i in df.columns:
    num += df[i][df[i]==0].count()
    
num

460

In [126]:
# normalising the data by dividing each row by it's max value
list_sim =[]
for i in range(k):
    temp_r = []
    div = df[i][i]
    for j in range(k):
        #print(df[j][i]/div)
        temp_r.append(df[j][i]/div)
    list_sim.append(temp_r)
    
df2 = pd.DataFrame(np.array(list_sim))
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1.000000,0.227239,0.224703,0.298818,0.258804,0.238386,0.247129,0.000000,0.326403,0.365367,0.200854,0.392893,0.224566,0.313720,0.325776,0.000000,0.281917,0.243244,0.000000,0.000000,0.253446,0.182742,0.248696,0.205268,0.185109,0.192088,0.000000,0.353271,0.189146,0.000000,0.214204,0.263202,0.200446,0.299093,0.221026,0.289391,0.183479,0.187091,0.000000,0.210256,0.000000,0.287844,0.190171,0.240487,0.254472,0.225738,0.000000,0.182829,0.208641,0.000000
1,0.335865,1.000000,0.270910,0.235427,0.270642,0.235409,0.294255,0.202769,0.334720,0.290434,0.236677,0.298989,0.264377,0.357218,0.219213,0.186542,0.286559,0.318299,0.209863,0.283365,0.299500,0.438671,0.203805,0.237475,0.221084,0.311715,0.218686,0.395040,0.288494,0.000000,0.310636,0.275392,0.418414,0.308055,0.292042,0.301962,0.181760,0.212894,0.294013,0.195290,0.198196,0.315879,0.220204,0.220221,0.173872,0.219637,0.189719,0.217157,0.200609,0.191650
2,0.215003,0.000000,1.000000,0.259321,0.236050,0.383329,0.271512,0.000000,0.276507,0.211056,0.000000,0.257052,0.000000,0.294464,0.254804,0.000000,0.000000,0.238880,0.000000,0.242456,0.240179,0.262263,0.302388,0.000000,0.000000,0.000000,0.000000,0.250187,0.238862,0.000000,0.206751,0.293471,0.000000,0.241565,0.247415,0.238427,0.000000,0.000000,0.000000,0.000000,0.000000,0.228379,0.212283,0.284583,0.222532,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.250772,0.000000,0.223502,1.000000,0.000000,0.000000,0.199682,0.000000,0.327534,0.291545,0.000000,0.278739,0.000000,0.000000,0.343620,0.000000,0.212006,0.000000,0.236340,0.000000,0.000000,0.000000,0.325715,0.221201,0.000000,0.000000,0.000000,0.208666,0.000000,0.000000,0.195559,0.210127,0.000000,0.259546,0.204526,0.200871,0.000000,0.236770,0.000000,0.220104,0.000000,0.208876,0.000000,0.201331,0.000000,0.000000,0.000000,0.196222,0.194765,0.000000
4,0.283464,0.238890,0.245555,0.183809,1.000000,0.277738,0.250272,0.000000,0.298835,0.187759,0.275882,0.289726,0.169072,0.342932,0.172016,0.161983,0.188744,0.255472,0.214665,0.000000,0.285807,0.234541,0.217629,0.000000,0.215106,0.223658,0.000000,0.248723,0.281893,0.166678,0.188805,0.261091,0.176468,0.241222,0.271241,0.210238,0.273744,0.167047,0.000000,0.169512,0.180579,0.331575,0.155795,0.240646,0.200745,0.301666,0.000000,0.236344,0.000000,0.000000
5,0.239019,0.000000,0.341272,0.195182,0.278349,1.000000,0.287127,0.000000,0.258552,0.195811,0.000000,0.238572,0.212615,0.267467,0.238856,0.000000,0.191416,0.268290,0.000000,0.233100,0.293897,0.226344,0.211190,0.000000,0.000000,0.203348,0.000000,0.239588,0.229906,0.000000,0.228457,0.280599,0.000000,0.228246,0.000000,0.000000,0.000000,0.000000,0.194429,0.224850,0.000000,0.231489,0.238335,0.281516,0.000000,0.000000,0.000000,0.203438,0.000000,0.200286
6,0.310221,0.271851,0.309837,0.236763,0.314561,0.384462,1.000000,0.000000,0.370665,0.246736,0.298138,0.290064,0.318123,0.316660,0.267080,0.000000,0.288610,0.211711,0.224455,0.289065,0.322374,0.341520,0.297099,0.236677,0.233386,0.322514,0.000000,0.294219,0.295891,0.000000,0.308500,0.326659,0.271267,0.310435,0.280848,0.000000,0.000000,0.000000,0.337207,0.219297,0.000000,0.338847,0.262115,0.357681,0.341645,0.268809,0.000000,0.258364,0.000000,0.238428
7,0.323952,0.229516,0.229863,0.302461,0.000000,0.203088,0.202113,1.000000,0.372226,0.447779,0.000000,0.346091,0.272103,0.211714,0.355358,0.254509,0.388172,0.221652,0.299871,0.327833,0.225218,0.304319,0.304855,0.376814,0.273701,0.295384,0.292328,0.317379,0.180297,0.209177,0.283117,0.335957,0.000000,0.378107,0.222468,0.349146,0.000000,0.409814,0.248621,0.364847,0.266774,0.344827,0.260843,0.333410,0.270857,0.000000,0.237123,0.415758,0.332705,0.216356
8,0.342231,0.260414,0.281726,0.368498,0.313536,0.273218,0.340472,0.260498,1.000000,0.398745,0.273679,0.373982,0.279747,0.318913,0.387588,0.237892,0.306988,0.238899,0.286457,0.265799,0.285464,0.317994,0.436045,0.247289,0.285460,0.297511,0.23129

In [127]:
#num of zeros in above df
num = 0
for i in df.columns:
    num += df2[i][df2[i]==0].count()
    #print(df2[i][df2[i]==0].count())
    
num

460

In [128]:
# making the matrix symmetric
count=0
for i in range(k):
    for j in range(k):
        if(i!=j):
            avg = (df2[i][j] + df2[j][i])/2
            df2[i][j] = avg
            df2[j][i] = avg
            #if(df2[j][i] ==0 and df2[i][j]==0):
            #    count += 1

df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1.000000,0.281552,0.219853,0.274795,0.271134,0.238702,0.278675,0.161976,0.334317,0.388110,0.213793,0.427243,0.261502,0.393406,0.336055,0.249988,0.264789,0.245199,0.111112,0.103188,0.260715,0.259942,0.243233,0.213828,0.226725,0.239058,0.169781,0.362529,0.198394,0.103210,0.260337,0.264870,0.236939,0.334729,0.230347,0.336157,0.204901,0.230588,0.119765,0.239005,0.155853,0.298515,0.223200,0.255510,0.273343,0.249112,0.000000,0.196241,0.244521,0.000000
1,0.281552,1.000000,0.135455,0.117714,0.254766,0.117704,0.283053,0.216143,0.297567,0.281255,0.220134,0.302493,0.132188,0.383130,0.109606,0.301845,0.266219,0.273268,0.206746,0.268609,0.149750,0.470183,0.101903,0.247173,0.228132,0.329961,0.250068,0.364836,0.295609,0.000000,0.302626,0.242911,0.404607,0.267738,0.240928,0.307122,0.186740,0.200842,0.296652,0.180696,0.286397,0.281142,0.211548,0.110111,0.178281,0.203760,0.270594,0.198555,0.193924,0.095825
2,0.219853,0.135455,1.000000,0.241412,0.240803,0.362301,0.290675,0.114931,0.279116,0.227209,0.116575,0.306101,0.000000,0.375300,0.251638,0.207389,0.101118,0.241714,0.117418,0.239129,0.120090,0.316344,0.293311,0.115452,0.115096,0.135442,0.131873,0.281127,0.258667,0.127147,0.254045,0.291975,0.128405,0.258205,0.256170,0.279041,0.090005,0.113796,0.139051,0.096886,0.173486,0.222057,0.238831,0.303480,0.244353,0.100487,0.146400,0.000000,0.096568,0.076156
3,0.274795,0.117714,0.241412,1.000000,0.091905,0.097591,0.218223,0.151230,0.348016,0.329984,0.099700,0.347389,0.000000,0.145175,0.375180,0.258702,0.215600,0.081360,0.287588,0.105054,0.000000,0.154020,0.347565,0.241077,0.105529,0.109912,0.123988,0.241230,0.109051,0.110142,0.243194,0.227386,0.085156,0.268678,0.203942,0.247119,0.066422,0.263318,0.107145,0.241213,0.185016,0.204372,0.119289,0.226480,0.122045,0.081298,0.144108,0.197384,0.231619,0.000000
4,0.271134,0.254766,0.240803,0.091905,1.000000,0.278044,0.282416,0.000000,0.306186,0.204496,0.330182,0.335647,0.084536,0.397494,0.086008,0.325876,0.206050,0.244522,0.245146,0.000000,0.142904,0.278773,0.216250,0.098652,0.260735,0.243864,0.115216,0.263776,0.301072,0.083339,0.212725,0.271978,0.185653,0.254851,0.275893,0.243959,0.292998,0.184812,0.100103,0.167966,0.263597,0.353955,0.077898,0.255609,0.232564,0.310882,0.000000,0.256291,0.000000,0.000000
5,0.238702,0.117704,0.362301,0.097591,0.278044,1.000000,0.335795,0.101544,0.265885,0.216405,0.141504,0.287331,0.106308,0.320447,0.254469,0.184562,0.198717,0.274064,0.129840,0.246139,0.146949,0.291964,0.215227,0.133917,0.116582,0.243602,0.120391,0.262998,0.272780,0.115749,0.266598,0.299625,0.093537,0.236869,0.105511,0.105981,0.112882,0.112383,0.223815,0.231796,0.177840,0.230157,0.261486,0.324591,0.128057,0.111598,0.146765,0.206228,0.091712,0.193342
6,0.278675,0.283053,0.290675,0.218223,0.282416,0.335795,1.000000,0.101057,0.355568,0.250813,0.310601,0.319206,0.159061,0.347845,0.253768,0.240769,0.258650,0.189219,0.242197,0.267121,0.161187,0.387307,0.264725,0.268303,0.240477,0.343361,0.144188,0.286319,0.294588,0.000000,0.309742,0.313732,0.298529,0.291372,0.259784,0.129040,0.100426,0.113974,0.350617,0.198156,0.169458,0.332921,0.261152,0.324300,0.338257,0.267114,0.000000,0.229968,0.104562,0.204778
7,0.161976,0.216143,0.114931,0.151230,0.000000,0.101544,0.101057,1.000000,0.316362,0.395942,0.000000,0.310958,0.136051,0.230587,0.302586,0.366457,0.294712,0.110826,0.268696,0.294583,0.112609,0.331971,0.255682,0.350681,0.264280,0.285334,0.301058,0.282277,0.183047,0.222027,0.282443,0.278662,0.000000,0.345906,0.190595,0.321394,0.000000,0.372092,0.234225,0.350825,0.335565,0.272098,0.236068,0.305783,0.239719,0.000000,0.318657,0.340782,0.291531,0.187984
8,0.334317,0.297567,0.279116,0.348016,0.306186,0.265885,0.355568,0.316362,1.000000,0.437117,0.278446,0.429277,0.139873,0.390593,0.394474,0.449673,0.305971,0.227370,0.325902,0.261767,0.142732,0.376664,0.422136,0.277477,0.306147,0.327920,0.28301

In [129]:
num = 0
for i in df.columns:
    num += df2[i][df2[i]==0].count()
    
num

192

In [130]:
df2.columns = result_q1
df2.to_csv('similarity_matrix.csv', index=False,mode='a')

In [145]:
#Building adjacency matrix for 63 OHSUMED queries


m = 1000

#looping over 63 UHSUMED queries
#for key in topk_results.keys():
for key in ['OHSU61','OHSU62','OHSU63']:
    #print(key)
    #fetching top k reults of each query
    result_q1 = topk_results[key]
    
    
    similarity_lists = []

    #for med_ui in result_q1 :
    for med_ui in result_q1:
        temp = {}
        temp_l = []
        query = index_data[med_ui]
        params = {
        'indent': 'true',
        'fl': ['medline_ui', 'score'],
        'q.op': 'OR',
        'rows': m,
        'q': query
        }

        response = requests.get(base_url, params=params)
        #print(r.json())

        # storing the score by looping over m results
        if response.json()['response']['numFound'] < m:
            for i in range(response.json()['response']['numFound']):
                temp[response.json()['response']['docs'][i]['medline_ui'][0]] = response.json()['response']['docs'][i]['score']
        else:    
            for i in range(m):
                temp[response.json()['response']['docs'][i]['medline_ui'][0]] = response.json()['response']['docs'][i]['score']

        #print(temp)

        # fetching the scores of top_k results of the query
        for r in result_q1:
            if r in temp.keys():
                print(r," ",temp[r])
                temp_l.append(temp[r])
            else:
                print(r," ",0)
                temp_l.append(0)

        # appending into a list
        similarity_lists.append(temp_l)

    #forming data frame
    df = pd.DataFrame(np.array(similarity_lists))
    
    # normalising the values of df
    list_sim =[]
    for i in range(k):
        temp_r = []
        div = df[i][i]
        for j in range(k):
            #print(df[j][i]/div)
            temp_r.append(df[j][i]/div)
        list_sim.append(temp_r)

    df2 = pd.DataFrame(np.array(list_sim))
    
    # making the df symmetric
    count=0
    for i in range(k):
        for j in range(k):
            if(i!=j):
                avg = (df2[i][j] + df2[j][i])/2
                df2[i][j] = avg
                df2[j][i] = avg
                #if(df2[j][i] ==0 and df2[i][j]==0):
                #    count += 1

    print(df2)
    
    df2.columns = result_q1
    df2.to_csv('similarity_matrix_for_graph.csv', index=False,mode='a')
    
    
    
    

89156302   189.48007
88207318   36.851986
91120638   0
87202778   0
91333929   24.639442
90178276   0
88103805   0
91088432   0
89344747   23.56366
90350732   0
91259536   0
89055695   0
90059789   0
91055012   0
90340679   0
87325521   0
91160766   0
89336620   0
91044091   24.46509
90191478   23.657024
89047608   0
90132154   0
90318686   0
90126601   0
90220124   59.63173
89171390   0
87140272   0
91069114   0
89182887   23.856827
90059788   0
87246351   0
88004376   0
88004301   0
90081204   0
91031245   0
91122337   0
91073036   0
88064644   0
91296287   0
91035821   0
91239268   23.981474
87171301   0
90377812   0
89270887   0
90299705   0
90025760   0
91281648   0
91096855   0
90072298   0
91368784   0
89156302   55.59616
88207318   182.87286
91120638   0
87202778   0
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732   0
91259536   0
89055695   0
90059789   0
91055012   0
90340679   0
87325521   0
91160766   0
89336620   0
91044091   0
90191478   0
890476

89156302   0
88207318   0
91120638   0
87202778   0
91333929   48.11332
90178276   0
88103805   0
91088432   30.378866
89344747   0
90350732   0
91259536   0
89055695   184.32008
90059789   0
91055012   0
90340679   0
87325521   0
91160766   0
89336620   0
91044091   42.11169
90191478   0
89047608   37.288197
90132154   0
90318686   0
90126601   0
90220124   0
89171390   0
87140272   0
91069114   35.240654
89182887   0
90059788   0
87246351   0
88004376   0
88004301   0
90081204   0
91031245   0
91122337   0
91073036   0
88064644   31.358763
91296287   31.526728
91035821   0
91239268   31.323954
87171301   30.884445
90377812   0
89270887   0
90299705   0
90025760   0
91281648   30.623816
91096855   0
90072298   0
91368784   0
89156302   0
88207318   0
91120638   0
87202778   0
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732   89.000824
91259536   0
89055695   0
90059789   549.8134
91055012   94.88278
90340679   0
87325521   0
91160766   0
89336620   0
91044091

89156302   68.596924
88207318   0
91120638   0
87202778   0
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732   74.05972
91259536   0
89055695   0
90059789   70.03089
91055012   76.77078
90340679   119.07518
87325521   98.82986
91160766   79.272194
89336620   119.36984
91044091   0
90191478   0
89047608   0
90132154   75.21566
90318686   344.4852
90126601   99.43416
90220124   0
89171390   81.3478
87140272   160.54027
91069114   0
89182887   82.86296
90059788   0
87246351   87.43258
88004376   81.43291
88004301   0
90081204   107.024124
91031245   91.47603
91122337   0
91073036   74.15264
88064644   0
91296287   0
91035821   99.81297
91239268   0
87171301   0
90377812   0
89270887   120.4569
90299705   75.09109
90025760   147.5714
91281648   0
91096855   73.20309
90072298   134.53543
91368784   134.29669
89156302   60.265915
88207318   60.059254
91120638   0
87202778   0
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732   82.82975
9125953

89156302   0
88207318   0
91120638   0
87202778   0
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732   0
91259536   0
89055695   0
90059789   0
91055012   0
90340679   61.176826
87325521   64.56502
91160766   0
89336620   69.20923
91044091   0
90191478   0
89047608   0
90132154   0
90318686   91.30262
90126601   59.996796
90220124   0
89171390   0
87140272   72.01838
91069114   0
89182887   0
90059788   0
87246351   61.472485
88004376   0
88004301   0
90081204   263.9121
91031245   53.365337
91122337   0
91073036   59.59953
88064644   0
91296287   0
91035821   61.31114
91239268   0
87171301   0
90377812   0
89270887   76.239174
90299705   0
90025760   81.58656
91281648   0
91096855   52.249355
90072298   82.27665
91368784   78.83229
89156302   0
88207318   0
91120638   0
87202778   102.42092
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732   0
91259536   0
89055695   0
90059789   0
91055012   0
90340679   0
87325521   0
91160766   0
893

89156302   97.58459
88207318   92.734955
91120638   0
87202778   0
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732   130.7868
91259536   0
89055695   0
90059789   96.62142
91055012   98.122536
90340679   89.690346
87325521   89.75433
91160766   0
89336620   0
91044091   0
90191478   83.39362
89047608   0
90132154   93.695244
90318686   108.88885
90126601   131.07552
90220124   84.75911
89171390   108.9564
87140272   109.4297
91069114   0
89182887   86.02006
90059788   96.06603
87246351   130.17378
88004376   104.756096
88004301   92.47833
90081204   94.95238
91031245   121.83886
91122337   0
91073036   96.593666
88064644   0
91296287   0
91035821   111.75129
91239268   0
87171301   0
90377812   92.1588
89270887   120.59491
90299705   476.83063
90025760   113.72262
91281648   0
91096855   119.8033
90072298   107.39278
91368784   99.67923
89156302   0
88207318   0
91120638   0
87202778   0
91333929   0
90178276   0
88103805   0
91088432   0
89344747   0
90350732

...

87241845   296.38715
89048603   46.051044
88109275   41.446144
88003498   37.996174
89185511   43.03688
90335453   49.122574
87313488   41.351707
89017893   41.093033
90040883   40.491894
91158215   0
89121520   45.168304
88199157   39.067917
91121407   41.26199
89241228   43.557228
88208122   39.303566
88210244   37.16822
91134337   0
91253539   42.747383
90144566   37.82438
90352519   0
88069214   0
88089752   41.894047
89058105   44.844795
87290547   37.63681
88103642   40.067936
88192141   0
87149335   0
87239465   0
91171136   0
88331735   0
90047634   43.964775
91189117   43.1786
88157120   42.20854
90304346   45.103558
87102376   37.41915
87297894   0
88145915   40.419785
90318184   0
90141006   0
88139473   37.168056
91247473   40.136303
89346964   0
91175208   0
91014999   0
89311327   0
91163911   0
91163912   0
91119036   0
91333994   0
91234294   0
87241845   53.27193
89048603   284.7478
88109275   63.449276
88003498   52.729412
89185511   49.6051
90335453   54.8988
8731348

87241845   41.07302
89048603   37.31986
88109275   35.08959
88003498   38.41485
89185511   34.486176
90335453   54.458008
87313488   53.67713
89017893   52.480083
90040883   37.209423
91158215   0
89121520   159.05492
88199157   38.270905
91121407   41.87542
89241228   51.889145
88208122   38.189075
88210244   38.949
91134337   42.353
91253539   45.63757
90144566   34.2272
90352519   0
88069214   45.211082
88089752   46.625374
89058105   45.294582
87290547   35.525597
88103642   0
88192141   0
87149335   0
87239465   0
91171136   0
88331735   0
90047634   48.84765
91189117   43.10376
88157120   52.853607
90304346   0
87102376   0
87297894   0
88145915   57.163486
90318184   0
90141006   32.652428
88139473   32.339878
91247473   42.171417
89346964   0
91175208   0
91014999   0
89311327   0
91163911   0
91163912   0
91119036   0
91333994   0
91234294   0
87241845   0
89048603   0
88109275   0
88003498   0
89185511   0
90335453   54.339573
87313488   0
89017893   0
90040883   0
91158215  

87241845   0
89048603   93.19341
88109275   0
88003498   109.03174
89185511   0
90335453   151.65811
87313488   105.298874
89017893   128.80595
90040883   0
91158215   0
89121520   126.94496
88199157   0
91121407   0
89241228   106.53644
88208122   111.039566
88210244   106.26982
91134337   116.551994
91253539   101.370834
90144566   86.369606
90352519   0
88069214   0
88089752   549.43677
89058105   106.50915
87290547   86.99759
88103642   0
88192141   0
87149335   0
87239465   0
91171136   0
88331735   0
90047634   92.590675
91189117   110.961716
88157120   137.50298
90304346   0
87102376   0
87297894   0
88145915   126.32021
90318184   0
90141006   91.000336
88139473   0
91247473   0
89346964   0
91175208   0
91014999   0
89311327   106.964096
91163911   0
91163912   0
91119036   0
91333994   0
91234294   0
87241845   40.555172
89048603   42.66512
88109275   0
88003498   35.611416
89185511   0
90335453   45.385376
87313488   46.613976
89017893   47.680267
90040883   0
91158215   0
8

87241845   67.82223
89048603   70.24829
88109275   0
88003498   76.641594
89185511   0
90335453   91.466095
87313488   102.32886
89017893   87.576164
90040883   0
91158215   0
89121520   83.80744
88199157   0
91121407   65.41425
89241228   65.77876
88208122   75.42932
88210244   78.11352
91134337   0
91253539   74.00004
90144566   0
90352519   0
88069214   63.60393
88089752   95.464226
89058105   70.57821
87290547   0
88103642   0
88192141   0
87149335   0
87239465   0
91171136   0
88331735   0
90047634   0
91189117   81.571785
88157120   391.8821
90304346   0
87102376   0
87297894   0
88145915   82.87038
90318184   0
90141006   0
88139473   0
91247473   65.52139
89346964   0
91175208   0
91014999   0
89311327   74.95557
91163911   0
91163912   0
91119036   0
91333994   0
91234294   0
87241845   78.85569
89048603   0
88109275   0
88003498   0
89185511   0
90335453   0
87313488   0
89017893   0
90040883   0
91158215   0
89121520   0
88199157   0
91121407   0
89241228   0
88208122   0
88

87241845   0
89048603   0
88109275   34.485973
88003498   0
89185511   0
90335453   34.607605
87313488   45.24052
89017893   0
90040883   35.0845
91158215   0
89121520   0
88199157   0
91121407   0
89241228   0
88208122   0
88210244   34.83884
91134337   0
91253539   30.939922
90144566   32.9877
90352519   0
88069214   32.3313
88089752   48.05791
89058105   32.17433
87290547   0
88103642   0
88192141   0
87149335   0
87239465   0
91171136   0
88331735   0
90047634   0
91189117   0
88157120   45.81158
90304346   0
87102376   0
87297894   0
88145915   0
90318184   0
90141006   0
88139473   0
91247473   33.252064
89346964   0
91175208   0
91014999   0
89311327   219.59476
91163911   0
91163912   0
91119036   0
91333994   0
91234294   0
87241845   0
89048603   0
88109275   0
88003498   0
89185511   0
90335453   0
87313488   0
89017893   0
90040883   0
91158215   38.596092
89121520   0
88199157   0
91121407   0
89241228   0
88208122   0
88210244   0
91134337   0
91253539   0
90144566   0
90